# Naive Baise Classifier

In [1]:
!pip install bs4
!pip install lxml
!pip install hazm

In [2]:
from os import listdir
from os.path import isfile, join

import bs4

In [7]:
mypath = 'Data'
dataset = [f for f in listdir(mypath) if isfile(join(mypath,f))]
dataset.remove('hamshahri.dtd')
dataset

['HAM2-070101.xml',
 'HAM2-070102.xml',
 'HAM2-070103.xml',
 'HAM2-070104.xml',
 'HAM2-070106.xml',
 'HAM2-070107.xml',
 'HAM2-070110.xml',
 'HAM2-070111.xml',
 'HAM2-070113.xml',
 'HAM2-070114.xml',
 'HAM2-070115.xml',
 'HAM2-070116.xml',
 'HAM2-070117.xml',
 'HAM2-070118.xml',
 'HAM2-070120.xml',
 'HAM2-070121.xml',
 'HAM2-070122.xml',
 'HAM2-070123.xml',
 'HAM2-070124.xml',
 'HAM2-070125.xml',
 'HAM2-070127.xml',
 'HAM2-070128.xml',
 'HAM2-070131.xml',
 'HAM2-070201.xml',
 'HAM2-070203.xml',
 'HAM2-070204.xml',
 'HAM2-070205.xml',
 'HAM2-070206.xml',
 'HAM2-070207.xml',
 'HAM2-070208.xml',
 'HAM2-070210.xml',
 'HAM2-070212.xml',
 'HAM2-070213.xml',
 'HAM2-070214.xml',
 'HAM2-070215.xml',
 'HAM2-070217.xml',
 'HAM2-070218.xml',
 'HAM2-070219.xml',
 'HAM2-070220.xml',
 'HAM2-070221.xml',
 'HAM2-070222.xml',
 'HAM2-070224.xml',
 'HAM2-070225.xml',
 'HAM2-070226.xml',
 'HAM2-070227.xml',
 'HAM2-070228.xml',
 'HAM2-070301.xml',
 'HAM2-070303.xml',
 'HAM2-070304.xml',
 'HAM2-070305.xml',


In [8]:
# 80% of dataset -> train_set 20% of dataset -> test_set
train_data = dataset[0:72]
test_data = dataset[72:]


# In case we want to apply Cross Validation

*Cross-Validation* is a validation technique designed to evaluate and assess how the results of statistical analysis (model) will generalize to an independent dataset. Cross-Validation is primarily used in scenarios where prediction is the main aim, and the user wants to estimate how well and accurately a predictive model will perform in real-world situations.

Cross-Validation seeks to define a dataset by testing the model in the training phase to help minimize problems like overfitting and underfitting. However, you must remember that both the validation and the training set must be extracted from the same distribution, or else it would lead to problems in the validation phase.

<img src = "https://i.stack.imgur.com/DlBMc.png" width=50%>

k = 5 


lenth_of_folds = len(dataset) / k


fold_1 = dataset[:lenth_of_folds]


fold_2 = dataset[lenth_of_folds:lenth_of_folds*2]


fold_3 = dataset[lenth_of_folds*2:lenth_of_folds*3]


fold_4 = dataset[lenth_of_folds*3:lenth_of_folds*4]


fold_5 = dataset[lenth_of_folds*4:]

In [11]:
train_dic = {}
for data in train_data:
    raw_file = open(join(mypath,data),'r',encoding='utf8')
    raw_file = raw_file.read()
    bs = bs4.BeautifulSoup(raw_file,'xml')
    

    for el in bs.findAll('DOC'):
        if el.TEXT: 
            body = el.TEXT.text
            for im in el.TEXT.findAll('IMAGE'):
                body = body.replace(im.text,'')
            temp_dic = {
                'title' : el.TITLE.text,
                'body' : body,
                'cat' : el.CAT.text
            }
            train_dic[el.DOCID.text] = temp_dic

len(train_dic)

7425

In [13]:
import pandas as pd
import hazm as hzm

normalizer = hzm.Normalizer()
vocabulary = []
COUNT = []

for key in train_dic.keys():
    tf_dict = {}
    normalized_title = normalizer.normalize(train_dic[key]['title'])
    tokenized_title = hzm.word_tokenize(normalized_title)
    normalized_body = normalizer.normalize(train_dic[key]['body'])
    tokenized_body = hzm.word_tokenize(normalized_body)
    tokens = tokenized_title + tokenized_body

    for token in tokens:
      vocabulary.append(token)
      if token in tf_dict.keys():
        tf_dict[token] += 1
      else:
        tf_dict[token] =1
    train_dic[key]['tf'] = tf_dict
    COUNT.append(len(tokens))


vocabulary = set(vocabulary)




ID = []
CLASS = []
TF = []
for key in train_dic.keys():
  ID.append(key)
  CLASS.append(train_dic[key]['cat'])
  TF.append(train_dic[key]['tf'])

DF_REF = {'ID':ID,'CLASS':CLASS,'tf':TF,'count':COUNT }


df = pd.DataFrame(DF_REF)

,ID,CLASS,tf,count
0,HAM2-851011-001,ادب و هنر,"{'مدیرکل': 1, 'کتاب': 23, 'و': 21, 'کتابخوانی'...",666
1,HAM2-851011-002,ادب و هنر,"{'مرگ': 2, 'هری': 7, 'پاتر': 7, 'و': 8, 'دردسر...",269
2,HAM2-851011-003,ادب و هنر,"{'تغییر': 1, 'در': 9, 'اجرای': 1, 'اسکار': 3, ...",236
3,HAM2-851011-004,ادب و هنر,"{'اخبار': 1, 'کوتاه': 1, 'چاپ': 6, '۲۴': 1, 'ش...",415
4,HAM2-851011-005,ادب و هنر,"{'مهران': 2, 'مدیری': 2, ':': 5, 'پا': 1, 'در'...",127
...,...,...,...,...
7420,HAM2-860129-114,گوناگون.شهری,"{'زاویه': 1, 'یک': 2, 'برنامه': 4, '،': 4, 'خط...",230
7421,HAM2-860129-115,علمی فرهنگی.علمی,"{'گوشت': 55, 'کمتر': 5, '،': 68, 'سلامت': 2, '...",1871
7422,HAM2-860129-116,علمی فرهنگی.علمی,"{'میوه': 12, 'و': 44, 'سبزی': 8, 'خواص': 1, 'آ...",777
7423,HAM2-860129-117,علمی فرهنگی.علمی,"{'جهان': 1, 'سلامت': 1, 'مرد': 3, 'مینیاتوری':...",903


In [14]:
categories = list(df['CLASS'].unique())

['ادب و هنر',
 'گوناگون',
 'اقتصاد.بورس و بانک',
 'ادب و هنر.هنر.سینما',
 'علمی فرهنگی.علمی.پزشکی و درمان',
 'اجتماعی',
 'اقتصاد',
 'گوناگون.حوادث',
 'علمی فرهنگی.علمی.ارتباطات و فناوری اطلاعات',
 'گوناگون.خارجی',
 'اجتماعی.معارف',
 'گوناگون.شهری',
 'سیاسی',
 'ادب و هنر.هنر.تئاتر',
 'ورزش',
 'ادب و هنر.هنر',
 'ادب و هنر.ادبیات',
 'علمی فرهنگی',
 'علمی فرهنگی.علمی.کتاب',
 'گردشگری',
 'ادب و هنر.هنر.موسیقی',
 'علمی فرهنگی.علمی',
 'سیاسی.سیاست داخلی']

In [15]:
# Calculate a class probability
def class_based_p(c):
  return len(df[df.CLASS==c])/len(df)*(10**3)

def word_based_p(w,c):
  tfs = list(df[df.CLASS == c]['tf'])
  cwc = 0

  for tf in tfs:
    if w in tf.keys():
      cwc+= tf[w]

  wc = df.groupby(['CLASS']).sum().loc[c].values[0]

  probability = (cwc+1)/(wc+len(vocabulary))  
  return probability*(10**3)

In [17]:
test_dic = {}
for data in test_data:
    raw_file = open(join(mypath,data),'r',encoding='utf8')
    raw_file = raw_file.read()
    bs = bs4.BeautifulSoup(raw_file,'xml')
    

    for el in bs.findAll('DOC'):
        if(el.TEXT):
            body = el.TEXT.text
            for im in el.TEXT.findAll('IMAGE'):
                body = body.replace(im.text,'')
            temp_dic = {
                'title' : el.TITLE.text,
                'body' : body,
                'cat' : el.CAT.text
            }
            test_dic[el.DOCID.text] = temp_dic

# len(test_dic)


1843

In [18]:
def predict(dic):
    normalizer = hzm.Normalizer()
    normalized_title = normalizer.normalize(dic['title'])
    tokenized_title = hzm.word_tokenize(normalized_title)
    normalized_body = normalizer.normalize(dic['body'])
    tokenized_body = hzm.word_tokenize(normalized_body)
    tokens = tokenized_title + tokenized_body
    probabilities = []

    for category in categories:

        total_p = class_based_p(category)
        for token in tokens:
            p = word_based_p(token,category)
            total_p*= p
        probabilities.append((category,total_p))

    probabilities.sort(reverse=True, key = lambda x: x[1])
    return probabilities[0][0],probabilities


In [22]:
# test_dic.keys()

dict_keys(['HAM2-860130-001', 'HAM2-860130-002', 'HAM2-860130-003', 'HAM2-860130-004', 'HAM2-860130-005', 'HAM2-860130-006', 'HAM2-860130-007', 'HAM2-860130-008', 'HAM2-860130-009', 'HAM2-860130-010', 'HAM2-860130-011', 'HAM2-860130-012', 'HAM2-860130-013', 'HAM2-860130-014', 'HAM2-860130-015', 'HAM2-860130-016', 'HAM2-860130-017', 'HAM2-860130-018', 'HAM2-860130-019', 'HAM2-860130-020', 'HAM2-860130-021', 'HAM2-860130-022', 'HAM2-860130-023', 'HAM2-860130-024', 'HAM2-860130-025', 'HAM2-860130-026', 'HAM2-860130-027', 'HAM2-860130-028', 'HAM2-860130-029', 'HAM2-860130-030', 'HAM2-860130-031', 'HAM2-860130-032', 'HAM2-860130-033', 'HAM2-860130-034', 'HAM2-860130-035', 'HAM2-860130-036', 'HAM2-860130-037', 'HAM2-860130-038', 'HAM2-860130-039', 'HAM2-860130-040', 'HAM2-860130-041', 'HAM2-860130-042', 'HAM2-860130-043', 'HAM2-860130-044', 'HAM2-860130-045', 'HAM2-860130-046', 'HAM2-860130-047', 'HAM2-860130-048', 'HAM2-860130-049', 'HAM2-860130-050', 'HAM2-860130-051', 'HAM2-860130-052', '

In [23]:
# dic = test_dic['HAM2-860130-074']

In [24]:
# dic['cat']

'گوناگون.شهری'

In [25]:
# cat, listed = predict(dic)

In [26]:
# listed

[('گوناگون.شهری', 79223007.72943),
 ('اجتماعی', 9.77268802693236e-19),
 ('گوناگون', 2.7872372745976955e-33),
 ('سیاسی', 1.0592224006867135e-41),
 ('اقتصاد', 2.7776782446326928e-43),
 ('ادب و هنر', 6.084617075513436e-48),
 ('علمی فرهنگی.علمی.ارتباطات و فناوری اطلاعات', 3.4184213166652556e-51),
 ('ادب و هنر.هنر.تئاتر', 2.5241110462329937e-59),
 ('علمی فرهنگی', 2.6705875079537854e-60),
 ('ادب و هنر.هنر.سینما', 4.7004100489960426e-80),
 ('ادب و هنر.هنر', 5.721448385429009e-83),
 ('علمی فرهنگی.علمی.پزشکی و درمان', 2.961852844155628e-88),
 ('اقتصاد.بورس و بانک', 4.1408817126536664e-89),
 ('گوناگون.خارجی', 7.001534197516966e-90),
 ('علمی فرهنگی.علمی.کتاب', 4.539885157520867e-96),
 ('علمی فرهنگی.علمی', 4.093160287018005e-96),
 ('گردشگری', 7.789253821364481e-107),
 ('گوناگون.حوادث', 5.066913474122132e-110),
 ('ادب و هنر.ادبیات', 1.3427930879927241e-129),
 ('ادب و هنر.هنر.موسیقی', 3.406100187433885e-137),
 ('سیاسی.سیاست داخلی', 4.870364559983155e-141),
 ('اجتماعی.معارف', 1.4113033068404063e-162)

# Classification

In [28]:
corrected = 0
wrong = 0
i=1
for key in test_dic.keys():
    print(f"Predicting the {key}  {i} out of {len(test_dic)} ")
    cat, listed = predict(test_dic[key])
    print(f'predicted cat is {cat}')
    if cat == test_dic[key]['cat']:
        print('Correct answer')
        corrected+=1
#     else:
#         wrong+=1
#         print('Wrong answer')
    i+=1
accuracy = corrected / len(test_dic)
print(f'Accuracy is {accuracy}')

Predicting the HAM2-860130-001  1 out of 1843 
predicted cat is ادب و هنر
Correct answer
Predicting the HAM2-860130-002  2 out of 1843 
predicted cat is گوناگون.شهری
Predicting the HAM2-860130-003  3 out of 1843 
predicted cat is ادب و هنر
Correct answer
Predicting the HAM2-860130-004  4 out of 1843 
predicted cat is گوناگون
Predicting the HAM2-860130-005  5 out of 1843 
predicted cat is ادب و هنر
Correct answer
Predicting the HAM2-860130-006  6 out of 1843 
predicted cat is گوناگون.شهری
Predicting the HAM2-860130-007  7 out of 1843 
predicted cat is ادب و هنر
Predicting the HAM2-860130-008  8 out of 1843 
predicted cat is علمی فرهنگی.علمی.پزشکی و درمان
Predicting the HAM2-860130-009  9 out of 1843 
predicted cat is گوناگون
Correct answer
Predicting the HAM2-860130-010  10 out of 1843 
predicted cat is گوناگون
Correct answer
Predicting the HAM2-860130-011  11 out of 1843 
predicted cat is گوناگون
Correct answer
Predicting the HAM2-860130-012  12 out of 1843 
predicted cat is اقتصاد
Pre

predicted cat is سیاسی
Correct answer
Predicting the HAM2-860130-092  92 out of 1843 
predicted cat is سیاسی
Correct answer
Predicting the HAM2-860130-093  93 out of 1843 
predicted cat is سیاسی
Correct answer
Predicting the HAM2-860130-094  94 out of 1843 
predicted cat is سیاسی
Correct answer
Predicting the HAM2-860130-095  95 out of 1843 
predicted cat is ادب و هنر
Predicting the HAM2-860130-096  96 out of 1843 
predicted cat is گوناگون.خارجی
Predicting the HAM2-860130-097  97 out of 1843 
predicted cat is سیاسی
Correct answer
Predicting the HAM2-860130-098  98 out of 1843 
predicted cat is گوناگون
Predicting the HAM2-860130-099  99 out of 1843 
predicted cat is ادب و هنر
Predicting the HAM2-860130-100  100 out of 1843 
predicted cat is ادب و هنر.هنر.سینما
Predicting the HAM2-860130-101  101 out of 1843 
predicted cat is ادب و هنر.هنر.سینما
Predicting the HAM2-860130-102  102 out of 1843 
predicted cat is ادب و هنر.هنر.سینما
Predicting the HAM2-860130-103  103 out of 1843 
predicted

predicted cat is گوناگون.شهری
Correct answer
Predicting the HAM2-860201-060  183 out of 1843 
predicted cat is گوناگون.شهری
Correct answer
Predicting the HAM2-860201-061  184 out of 1843 
predicted cat is گوناگون.شهری
Correct answer
Predicting the HAM2-860201-062  185 out of 1843 
predicted cat is اقتصاد
Predicting the HAM2-860201-063  186 out of 1843 
predicted cat is سیاسی
Correct answer
Predicting the HAM2-860201-064  187 out of 1843 
predicted cat is سیاسی
Correct answer
Predicting the HAM2-860201-065  188 out of 1843 
predicted cat is سیاسی
Correct answer
Predicting the HAM2-860201-066  189 out of 1843 
predicted cat is سیاسی
Correct answer
Predicting the HAM2-860201-067  190 out of 1843 
predicted cat is سیاسی
Correct answer
Predicting the HAM2-860201-068  191 out of 1843 
predicted cat is سیاسی
Correct answer
Predicting the HAM2-860201-069  192 out of 1843 
predicted cat is سیاسی
Correct answer
Predicting the HAM2-860201-070  193 out of 1843 
predicted cat is سیاسی
Correct answe

predicted cat is سیاسی
Predicting the HAM2-860202-059  274 out of 1843 
predicted cat is گوناگون.خارجی
Correct answer
Predicting the HAM2-860202-060  275 out of 1843 
predicted cat is گوناگون.شهری
Correct answer
Predicting the HAM2-860202-061  276 out of 1843 
predicted cat is گوناگون.شهری
Correct answer
Predicting the HAM2-860202-062  277 out of 1843 
predicted cat is گوناگون.شهری
Correct answer
Predicting the HAM2-860202-063  278 out of 1843 
predicted cat is گوناگون.شهری
Correct answer
Predicting the HAM2-860202-064  279 out of 1843 
predicted cat is گوناگون.شهری
Correct answer
Predicting the HAM2-860202-065  280 out of 1843 
predicted cat is ادب و هنر.هنر.تئاتر
Predicting the HAM2-860202-066  281 out of 1843 
predicted cat is اقتصاد
Predicting the HAM2-860202-067  282 out of 1843 
predicted cat is سیاسی
Correct answer
Predicting the HAM2-860202-068  283 out of 1843 
predicted cat is سیاسی
Correct answer
Predicting the HAM2-860202-069  284 out of 1843 
predicted cat is سیاسی
Correct

KeyboardInterrupt: 

@author : 

A.Hossein Dadashzade - M.Hossein Hashemi

In [29]:
accuracy = corrected / 308
print(f'Accuracy is {accuracy}')

Accuracy is 0.7467532467532467
